In [2]:
from gamspy import (
    Container,Set,Alias,Parameter,Variable,Equation,Model,Problem,Sense,Options,
    Domain,Number,Sum,Product,Smax,Smin,Ord,Card,ModelStatus,SpecialValues
)

In [10]:
import numpy as np
from gamspy import (
    Container, Set, Parameter, Equation
)

# Initialize GAMS container
m = Container()

# Set of months from January (1) to December (12)
months = Set(m, 'months', records=[str(i + 1) for i in range(12)])
# Parameters for monthly expenses
living = 550
rent = 650
car = 340
leisure_pay = 165
total = living + rent + car

allowance = 150
dentistry = 1900

leisure = Variable(m, 'leisure', domain=[months], description='Monthly amount spend on leisure')
enjoyment = []

for month in range(1, 13):

    if month % 2 == 1:
        total += 135
    if month % 4 == 1:
        total += 100
    if month % 6 == 1:
        total += 850
    
    e = month * (13 - month) * leisure_pay
    enjoyment.append(e)
# Parameters for enjoyment and bills
billpaid = Parameter(m, 'billpaid', records=total)

enjoy = Parameter(m, 'enjoy', domain=[months], records = np.array(enjoyment))

budget_constraint = Equation(m, 'budget_constraint', domain=[months])
budget_constraint[months] = (
    leisure[months] + billpaid <= dentistry + allowance
)

leisure_minimum = Equation(m, 'leisure_minimum', domain=[months])
leisure_minimum[months] = leisure[months] >= 165


budget_model = Model(
    m,
    name='budget_model',
    equations=[budget_constraint, leisure_minimum],
    problem=Problem.LP,
    sense=Sense.MAX,
    objective=Sum([months], enjoy[months])  # Maximize the total enjoyment
)

# Solve the model
budget_model.solve()



,Solver Status,Model Status,Objective,Num of Equations,Num of Variables,Model Type,Solver,Solver Time
0,Normal,InfeasibleNoSolution,NA,25,13,LP,CPLEX,0.001
